# Mistral TGI Real-Time Deployment

In this example we will take a look at deploying Mistral7B via the [Text Generation Inference (TGI)](https://github.com/huggingface/text-generation-inference) container on SageMaker Real-Time Inference. For further Load Testing and AutoScaling setup please refer to the LMI example within the directory. In this example we take the SageMaker deployment code directly from the [HuggingFace Model Hub for Mistral 7B](https://huggingface.co/mistralai/Mistral-7B-v0.1).

## Setup

In [ ]:
import json
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

## Endpoint Deployment

In [ ]:
# Hub Model configuration. https://huggingface.co/models
hub = {
	'HF_MODEL_ID':'mistralai/Mistral-7B-v0.1',
	'SM_NUM_GPUS': json.dumps(1) # can increase this value if using a larger instance for deployment
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	image_uri=get_huggingface_llm_image_uri("huggingface",version="1.1.0"),
	env=hub,
	role=role, 
)

In [ ]:
# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1,
	instance_type="ml.g5.2xlarge",
	container_startup_health_check_timeout=300,
  )

## Sample Inference

In [ ]:
# send request
predictor.predict({
	"inputs": "My name is Julien and I like to",
})

## Cleanup

In [ ]:
predictor.delete_endpoint()